# Treadmill experiments

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
project = "colasa_3d-vision_revisions"

In [ ]:
import flexiznam as flz

flm_sess = flz.get_flexilims_session(project_id=project)
mice = flz.get_entities(datatype='mouse', flexilims_session=flm_sess)
print(f"{len(mice)} mice in {project}")

In [ ]:
protocol_base = 'SpheresTubeMotor'

treadmill_sessions = {}
for mouse_name, mouse_data in mice.iterrows():
    sessions = flz.get_children(parent_id=mouse_data.id, flexilims_session=flm_sess, children_datatype='session')
    for session_name, sess_data in sessions.iterrows():
        recordings = flz.get_children(parent_id=sess_data.id, flexilims_session=flm_sess, children_datatype='recording')
        if protocol_base in recordings.protocol.values:
            treadmill_sessions[session_name] = recordings
print(f"{len(treadmill_sessions)} sessions with treadmill data")

In [ ]:
protocol_base = 'SpheresTubeMotor'
for session_name, recordings in treadmill_sessions.items():
    # Get the recordings that are not more
    recordings = recordings[recordings.protocol != protocol_base]
    print(f"{len(recordings)} sphere protocols")
    break
    

In [ ]:
import flexiznam as flz
from cottage_analysis.analysis.spheres import *
import flexiznam as flz

exclude_datasets = None

harp_is_in_recording = True
use_onix = False
conflicts = "skip"
sync_kwargs = None
ephys_kwargs = None
# We can just run the same pipeline. It will skip depth and rsof fit and just run the
# the rf fit
protocol_base = "SpheresPermTubeReward"
flexilims_session = flz.get_flexilims_session(project_id=project)
assert flexilims_session is not None or project is not None
filter_datasets = {"anatomical_only": 3}
recording_type = "two_photon"
photodiode_protocol = 5
return_volumes = True


load_onix = False if recording_type == "two_photon" else True
for i, recording_name in enumerate(recordings.name):
    print(f"Processing recording {i+1}/{len(recordings)}")
    recording, harp_recording, onix_rec = get_relevant_recordings(
        recording_name, flexilims_session, harp_is_in_recording, load_onix
    )        
    vs_df = synchronisation.generate_vs_df(
        recording=recording,
        photodiode_protocol=photodiode_protocol,
        flexilims_session=flexilims_session,
        harp_recording=harp_recording,
        onix_recording=onix_rec if use_onix else None,
        project=project,
        conflicts=conflicts,
        sync_kwargs=sync_kwargs,
        protocol_base=protocol_base,
    )
    imaging_df = synchronisation.generate_imaging_df(
        vs_df=vs_df,
        recording=recording,
        flexilims_session=flexilims_session,
        filter_datasets=filter_datasets,
        exclude_datasets=exclude_datasets,
        return_volumes=return_volumes,
    )
    imaging_df = format_imaging_df(imaging_df=imaging_df, recording=recording)

In [ ]:
imaging_df.columns

In [ ]:
recording_name

In [ ]:
harp_recording

In [ ]:
vis_stim_ds = flz.get_datasets(origin_id=harp_recording.id, flexilims_session=flm_sess, dataset_type='visstim', allow_multiple=False)
vis_stim_ds

In [ ]:
vis_stim_ds.extra_attributes

In [ ]:
s = vis_stim_ds.path_full / vis_stim_ds.extra_attributes['csv_files']['FrameLog']
frame_df = pd.read_csv(s)
frame_df.head()